# Where do I walk?
## Using RAPIDS to determine the shortest walking distance

## Load the modules

In [129]:
import cudf
import cuspatial
import nvstrings
import nvtext
from collections import OrderedDict
import numpy as np
import datetime as dt
import cugraph

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.34 ms


## Read in the data

In [130]:
dtypes = OrderedDict([
    ('OccupancyDateTime', 'date'),
    ('PaidOccupancy', 'int64'),
    ('BlockfaceName', 'str'),
    ('SideOfStreet', 'str'),
    ('SourceElementKey', 'int64'),
    ('ParkingTimeLimitCategory', 'int64'),
    ('ParkingSpaceCount', 'int64'),
    ('PaidParkingArea', 'str'),
    ('PaidParkingSubArea', 'str'),
    ('PaidParkingRate', 'int8'),
    ('ParkingCategory', 'str'),
    ('Location', 'str'),
    ('dow', 'int8')
])

df = cudf.read_csv(
    '../data/parking_MayJun2019.csv'
    , skiprows=1
    , dtype=list(dtypes.values())
    , names=list(dtypes.keys())
)

df = df[['SourceElementKey', 'Location']]

time: 6.07 s


In [131]:
def extractLon(location):
    lon = location.str.extract('([0-9\.\-]+) ([0-9\.]+)')[0]
    return lon.str.stod()

def extractLat(location):
    lon = location.str.extract('([0-9\.\-]+) ([0-9\.]+)')[1]
    return lon.str.stod()
    

time: 669 µs


In [132]:
locations = df.drop_duplicates()
del df

locations['longitude'] = extractLon(locations['Location'])
locations['latitude'] = extractLat(locations['Location'])
locations = locations[['SourceElementKey', 'longitude', 'latitude']]

time: 3.14 s


In [133]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="todrabas_test")
location = geolocator.geocode("400 Broad St, Seattle, WA 98109") # SPACE NEEDLE
# location = geolocator.geocode("85 Pike St, Seattle, WA 98101") # SPACE NEEDLE

location
# locations['LON_Ref'] = location.longitude
# locations['LAT_Ref'] = location.latitude

Location(Space Needle, 400, Broad Street, South Lake Union, Belltown, Seattle, King County, Washington, 98109, USA, (47.6205131, -122.349303598832, 0.0))

time: 730 ms


## As crow flies vs as people walk

### Read in the graph data
Thanks to John Murray for analyzing the map of King County roads and producing the data we will now use.

#### Download and unpack the data

In [7]:
import os

directory  = os.path.exists('../data')
archive    = os.path.exists('../data/king_county_road_graph_20190909.tar.gz')
file_graph = os.path.exists('../data/king_county_road_graph_20190909.csv')
file_nodes = os.path.exists('../data/king_county_road_nodes_20190909.csv')

if not directory:
    os.mkdir('../data')

if not archive:
    import wget, shutil
    
    wget.download('http://tomdrabas.com/data/seattle_parking/king_county_road_graph_20190909.tar.gz')
    shutil.move('king_county_road_graph_20190909.tar.gz', '../data/king_county_road_graph_20190909.tar.gz')
    
if not file_graph or not file_nodes:
    import tarfile

    tf = tarfile.open('../data/king_county_road_graph_20190909.tar.gz')
    tf.extractall(path='../data/')

time: 2.34 ms


In [8]:
!head ../data/king_county_road_graph_20190909.csv

node1,node2,LENGTH
89108,27652,5.02825
27652,89108,5.02825
27652,122930,112.417
122930,27652,112.417
36778,36779,48.2475
36779,36778,48.2475
26559,26559,48.3425
26559,26559,48.3425
2634,78382,372.325
time: 515 ms


In [9]:
!head ../data/king_county_road_nodes_20190909.csv

NodeID,Lon,Lat
1,-121.431505,47.331052
2,-121.430642,47.334726
3,-121.404812,47.288757
4,-121.408953,47.289157
5,-121.432249,47.292963
6,-121.432702,47.294297
7,-121.432724,47.329542
8,-121.367462,47.290593
9,-121.360285,47.29045
time: 544 ms


#### Let's read the road data

In [134]:
road_graph_data = cudf.read_csv('../data/king_county_road_graph_20190909.csv')
road_graph_data['node1'] = road_graph_data['node1'].astype('int32')
road_graph_data['node2'] = road_graph_data['node2'].astype('int32')
road_graph_data['LENGTH'] = road_graph_data['LENGTH'] * 3 # convert to feet as the LENGHT was given in yards

time: 16 ms


In [135]:
road_nodes = cudf.read_csv('../data/king_county_road_nodes_20190909.csv')
road_nodes['NodeID'] = road_nodes['NodeID'].astype('int32')

time: 9.08 ms


Store the maximum of the `NodeID` so we can later append the parking nodes.

In [136]:
offset = 100000
nodeId = road_nodes['NodeID'].max()                       ## so we can number the parking nodes properly (since we'll be adding a perpendicular projections)
parking_nodes_idx = road_nodes['NodeID'].max() + offset   ## retain it so we can later filter the results to only parking locations
nodeId

127380

time: 2.46 ms


Move all the parking locations to host (via `.to_pandas()` method) so we can loop through all the ~1500 parking locations. Here, we also create an empty DataFrame that will hold the parking location nodes.

In [137]:
parking_locations = locations.to_pandas().to_dict('records')
parking_locations_nodes = cudf.DataFrame(columns=['NodeID', 'Lon', 'Lat', 'SourceElementKey'])
added_location_edges    = cudf.DataFrame(columns=['node1', 'node2', 'LENGTH'])

time: 194 ms


Let's process the parking data. In this naive approach we simply search for 3 closest road intersections to the parking location. We will then add two edges to the overall graph that link the parking node to the nearest intersection. This approach currently is producing some artifacts but we will deal with this in the future; in this example we want to simply explore the efficacy of finding the shortest path between the nodes in the graph.

In [138]:
def kernel_find_projection(Lon_x, Lat_x, Lon_y, Lat_y, Lon_PROJ, Lat_PROJ, Lon_REF, Lat_REF):
    for i, (lon_x, lat_x, lon_y, lat_y) in enumerate(zip(Lon_x, Lat_x, Lon_y, Lat_y)):
        # special case where A and B have the same LON i.e. vertical line
        if lon_x == lon_y:
            Lon_PROJ[i] = lon_x
            Lat_PROJ[i] = Lat_REF    
        else:
            # find slope
            a_xy = (lat_x - lat_y) / float(lon_x - lon_y)

            # special case where A and B have the same LAT i.e. horizontal line
            if a_xy == 0:
                Lon_PROJ[i] = Lon_REF
                Lat_PROJ[i] = lat_x
            else: 
                # if neither of the above special cases apply
                # find the equation of the perpendicular line
                a_R  = -1 / a_xy                    ### SLOPE

                # find intersections
                b_xy = lat_x - a_xy * lon_x
                b_R  = Lat_REF - a_R  * Lon_REF

                # find the coordinates
                Lon_PROJ[i] = (b_R - b_xy) / (a_xy - a_R)
                Lat_PROJ[i] = a_R * Lon_PROJ[i] + b_R

time: 6.77 ms


In [139]:
[e for e in parking_locations if e['SourceElementKey'] == 68910]

[{'SourceElementKey': 68910,
  'longitude': -122.33855514999998,
  'latitude': 47.6073621}]

time: 2.42 ms


In [140]:
parking_locations_cnt = len(parking_locations)
print('Number of parking locations: {0:,}'.format(parking_locations_cnt))

for i, loc in enumerate(parking_locations):
    if i % 100 == 0:
        print('Processed: {0:,} ({1:2%}) nodes'.format(i, i/float(parking_locations_cnt)))
    nodeId = nodeId + 1
    lat_r = loc['latitude']
    lon_r = loc['longitude']

    paths = (
        road_graph_data
        .rename({'node1': 'NodeID'})
        .merge(road_nodes[['NodeID', 'Lat', 'Lon']], on='NodeID', how='left')
        .rename({'NodeID': 'node1', 'node2': 'NodeID'})
        .merge(road_nodes[['NodeID', 'Lat', 'Lon']], on='NodeID', how='left')
        .rename({'NodeID': 'node2'})
        .query('Lat_x >= (@lat_r - 0.005) and Lat_x <= (@lat_r + 0.005)')
        .query('Lon_x >= (@lon_r - 0.005) and Lon_x <= (@lon_r + 0.005)')
        .query('Lat_y >= (@lat_r - 0.005) and Lat_y <= (@lat_r + 0.005)')
        .query('Lon_y >= (@lon_r - 0.005) and Lon_y <= (@lon_r + 0.005)')
    )
#     print(len(paths))


    paths['Lon_REF'] = loc['longitude']
    paths['Lat_REF'] = loc['latitude']

    paths = paths.apply_rows(
        kernel_find_projection
        , incols  = ['Lon_x', 'Lat_x', 'Lon_y', 'Lat_y', 'Lon_REF', 'Lat_REF']
        , outcols = {'Lon_PROJ': np.float64, 'Lat_PROJ': np.float64}
        , kwargs  = {'Lon_REF': loc['longitude'], 'Lat_REF': loc['latitude']}
    )


    paths['Length_x_PROJ'] = cuspatial.haversine_distance(
              paths['Lon_x']
            , paths['Lat_x']
            , paths['Lon_PROJ']
            , paths['Lat_PROJ']) * 0.621371 * 5280

    paths['Length_y_PROJ'] = cuspatial.haversine_distance(
              paths['Lon_y']
            , paths['Lat_y']
            , paths['Lon_PROJ']
            , paths['Lat_PROJ']) * 0.621371 * 5280

    paths['Length_REF_PROJ'] = cuspatial.haversine_distance(
              paths['Lon_REF']
            , paths['Lat_REF']
            , paths['Lon_PROJ']
            , paths['Lat_PROJ']) * 0.621371 * 5280
    
#     paths['Length_REF_X'] = cuspatial.haversine_distance(
#               paths['Lon_x']
#             , paths['Lat_x']
#             , paths['Lon_REF']
#             , paths['Lat_REF']) * 0.621371 * 5280
    
#     paths['Length_REF_Y'] = cuspatial.haversine_distance(
#               paths['Lon_y']
#             , paths['Lat_y']
#             , paths['Lon_REF']
#             , paths['Lat_REF']) * 0.621371 * 5280

    paths['PROJ_between'] = (paths['Length_x_PROJ'] + paths['Length_y_PROJ']) <= (paths['LENGTH'] + 1)
#     paths['PROJ_between_test'] = paths['PROJ_between']  <= (paths['LENGTH'] + 1)
    
#     print(paths.sort_values(by='Length_REF_X').to_pandas())
    closest = paths.query('PROJ_between').nsmallest(1, 'Length_REF_PROJ').to_pandas().to_dict('records')[0]
#     print(closest)

    # add nodes
    nodes =    cudf.DataFrame({
          'NodeID': [nodeId + offset, nodeId]
        , 'Lon':    [closest['Lon_REF'], closest['Lon_PROJ']]
        , 'Lat':    [closest['Lat_REF'], closest['Lat_PROJ']]
        , 'SourceElementKey': [loc['SourceElementKey'], None]
    })

    parking_locations_nodes = cudf.concat([parking_locations_nodes, nodes])

    # add edges (bi-directional)
    edges = cudf.DataFrame({
          'node1':  [nodeId, nodeId, nodeId, closest['node1'], closest['node2'], nodeId + offset]
        , 'node2':  [closest['node1'], closest['node2'], nodeId + offset, nodeId, nodeId, nodeId]
        , 'LENGTH': [
              closest['Length_x_PROJ'], closest['Length_y_PROJ'], closest['Length_REF_PROJ']
            , closest['Length_x_PROJ'], closest['Length_y_PROJ'], closest['Length_REF_PROJ']
        ]
    })

    added_location_edges = cudf.concat([added_location_edges, edges]) ## append to the temp DataFrame

print('Finished processing...')

Number of parking locations: 1,473
Processed: 0 (0.000000%) nodes
Processed: 100 (6.788866%) nodes
Processed: 200 (13.577733%) nodes
Processed: 300 (20.366599%) nodes
Processed: 400 (27.155465%) nodes
Processed: 500 (33.944331%) nodes
Processed: 600 (40.733198%) nodes
Processed: 700 (47.522064%) nodes
Processed: 800 (54.310930%) nodes
Processed: 900 (61.099796%) nodes
Processed: 1,000 (67.888663%) nodes
Processed: 1,100 (74.677529%) nodes
Processed: 1,200 (81.466395%) nodes
Processed: 1,300 (88.255261%) nodes
Processed: 1,400 (95.044128%) nodes
Finished processing...
time: 3min 21s


Now we can find the nearest intersections from the Kerry Park!

In [141]:
road_nodes = (
    cudf
    .concat([road_nodes[['NodeID', 'Lon', 'Lat']], parking_locations_nodes])
    .reset_index(drop=True)
)

time: 25.5 ms


In [142]:
road_nodes['Lon_REF'] = location.longitude
road_nodes['Lat_REF'] = location.latitude

road_nodes['Distance'] = cuspatial.haversine_distance(
          road_nodes['Lon']
        , road_nodes['Lat']
        , road_nodes['Lon_REF']
        , road_nodes['Lat_REF']) * 0.621371 * 5280

space_needle_to_nearest_intersection = road_nodes.nsmallest(5, 'Distance') ### Space Needle is surrounded by around 5 road intersections hence we add 5
space_needle_to_nearest_intersection_dist = space_needle_to_nearest_intersection['Distance'].to_array()[0]

space_needle_to_nearest_intersection['node1'] = nodeId + 2
space_needle_to_nearest_intersection = (
    space_needle_to_nearest_intersection
    .rename({'NodeID': 'node2', 'Distance': 'LENGTH'})
    [['node1', 'node2', 'LENGTH']]
)

road_graph_data = cudf.concat([space_needle_to_nearest_intersection, added_location_edges, road_graph_data])
space_needle_to_nearest_intersection ### SHOW THE EDGES

,node1,node2,LENGTH
47756,128855,47757,175.906391
80448,128855,80449,200.062128
96739,128855,96740,261.056715
108797,128855,108798,277.221141
47827,128855,47828,301.715490


time: 77.6 ms


### The road graph

In [143]:
road_graph_data = road_graph_data.reset_index(drop=True)
road_graph_data['node1'] = road_graph_data['node1'].astype('int32')
road_graph_data['node2'] = road_graph_data['node2'].astype('int32')

sources      = cudf.Series(road_graph_data['node1'])
destinations = cudf.Series(road_graph_data['node2'])
distances    = cudf.Series(road_graph_data['LENGTH'])

g = cugraph.Graph()
g.add_edge_list(sources, destinations, distances)

time: 15.1 ms


Now we can use the `.sssp(...)` method from `cugraph` to find the shortest distances to parking spots from the Space Needle!

In [144]:
all_distances = cugraph.sssp(g, nodeId + 2)
distances = all_distances.query('vertex > @parking_nodes_idx and distance < 1000')
# all_distances.query('distance < 1000')
distances

,vertex,distance,predecessor
227585,227585,494.541097,127585
227586,227586,471.843339,127586
227666,227666,978.714937,127666
227667,227667,986.632352,127667
227822,227822,978.340665,127822
227823,227823,954.882271,127823
227912,227912,796.521774,127912
227913,227913,793.130111,127913
228160,228160,587.108025,128160
228161,228161,542.478604,128161


time: 77.9 ms


`cugraph` returns a DataFrame with vertex, distance to that vertex, and the total distance traveled to that vertex from the `nodeId + 1` node -- the Space Needle. Here, we unfold the full path.

In [145]:
# unfold -- create the whole path
closest_node = nodeId + 2
parking_cnt = distances['vertex'].count()

for i in range(parking_cnt):
    print('Processing record: {0}'.format(i))
    parking_node = distances.iloc[i]
    vertex = int(parking_node[0])
    predecessor = int(parking_node[2])
    
    if i == 0:
        paths = all_distances.query('vertex == @vertex')
    else:
        paths = cudf.concat([all_distances.query('vertex == @vertex'), paths])

    while vertex != closest_node:
        temp = all_distances.query('vertex == @predecessor')
        paths = cudf.concat([temp, paths])
        predecessor = temp['predecessor'].to_array()[0]
        vertex = temp['vertex'].to_array()[0]

Processing record: 0
Processing record: 1
Processing record: 2
Processing record: 3
Processing record: 4
Processing record: 5
Processing record: 6
Processing record: 7
Processing record: 8
Processing record: 9
Processing record: 10
Processing record: 11
time: 880 ms


### Charting the paths

In [146]:
paths['vertex'] = paths['vertex'].astype('int64')
paths['predecessor'] = paths['predecessor'].astype('int64')
paths = paths.drop_duplicates()

### process the data so we get the Lat/Lon back for both src and dest
### then move to host
paths_host = (
    paths
    .rename({'vertex': 'NodeID'})
    .merge(road_nodes[['NodeID', 'Lat', 'Lon']], on='NodeID', how='left')
    .rename({'NodeID': 'vertex', 'predecessor': 'NodeID'})
    .merge(road_nodes[['NodeID', 'Lat', 'Lon']], on='NodeID', how='left')
    .fillna({'Lat_y': location.latitude, 'Lon_y': location.longitude})
    [['vertex', 'Lat_x', 'Lon_x', 'Lat_y', 'Lon_y']]
    .query('vertex != @nodeId + 2')
    .to_pandas()
)

paths_host

,vertex,Lat_x,Lon_x,Lat_y,Lon_y
0,228161,47.619145,-122.348791,47.619151,-122.348892
2,228224,47.617998,-122.349296,47.618056,-122.349340
3,228506,47.620150,-122.347713,47.620153,-122.347591
4,47756,47.620906,-122.347603,47.620906,-122.348736
5,47758,47.620714,-122.347605,47.620906,-122.347603
6,47797,47.620905,-122.347358,47.620906,-122.347603
7,47799,47.620901,-122.346298,47.620905,-122.347358
8,47829,47.619735,-122.347581,47.619738,-122.348174
9,47830,47.618574,-122.348924,47.619056,-122.348898
10,47832,47.618390,-122.348900,47.618574,-122.348924


time: 48.4 ms


Get the information about the parking spots so we can create info boxes.

In [147]:
distances['vertex'] = distances['vertex'].astype('int64')
distances_host = (
    distances
    .rename({'vertex': 'NodeID'})
    .merge(road_nodes[['NodeID', 'Lat', 'Lon', 'SourceElementKey']], on='NodeID')
    [['SourceElementKey', 'Lat', 'Lon', 'distance']]
    .to_pandas()
#     .to_dict('records')
)

distances_host

,SourceElementKey,Lat,Lon,distance
0,54986,47.617998,-122.349296,982.444003
1,11133,47.619804,-122.348174,494.541097
2,11134,47.619679,-122.348258,471.843339
3,13129,47.620966,-122.345670,978.714937
4,13130,47.620826,-122.345645,986.632352
5,51493,47.619064,-122.349040,587.108025
6,51494,47.619145,-122.348791,542.478604
7,75173,47.620150,-122.347713,783.243667
8,33741,47.619808,-122.346958,796.521774
9,33742,47.619666,-122.346963,793.130111


time: 27.2 ms


In [148]:
info_box_template = """
<dl>
<dt><dd>SourceElementKey</dd><dd>{SourceElementKey}</dd></dt>
<dt><dd>Distance        </dd><dd>{distance:.0f} ft.</dd></dt>
</dl>
"""

parking_info = [info_box_template.format(**parking) for parking in distances_host.to_dict('records')]

time: 4.47 ms


And... plot!

In [150]:
import gmaps
# import gmaps.datasets
gmaps.configure(api_key="---->>>> YOUR API KEY HERE <<<<----") # Your Google API key, go to https://console.developers.google.com

parking_layer = gmaps.symbol_layer(
    distances_host[['Lat', 'Lon']], fill_color="green", stroke_color="green", scale=3, info_box_content=parking_info
)

destinations_layer = gmaps.symbol_layer(
    [[location.latitude, location.longitude]]
    , info_box_content=['DESTINATION']
    , scale=5
    , fill_color="red"
    , stroke_color="red"
)

lines_layer = gmaps.drawing_layer(features=[
    gmaps.Line(
          start = (path['Lat_x'], path['Lon_x'])
        , end   = (path['Lat_y'], path['Lon_y'])
        , stroke_weight=2
        , stroke_color="red"
    )
    for path in paths_host.to_dict('records')]
)

fig = gmaps.figure(layout={'height': '500px'})
fig.add_layer(parking_layer)
fig.add_layer(destinations_layer)
fig.add_layer(lines_layer)
fig

Figure(layout=FigureLayout(height='500px'))

time: 95.6 ms
